# Relatório da disciplina de Algebra Linear Computacional (CKP8122)
 - Aluno: Madson Luiz Dantas Dias
 - Professor: Creto Augusto Vidal

## Lista de implementações
### métodos de mínimos quadrados
 10. Gram-Schmidt
 11. decomposição QR usando householder

In [1]:
from Matrix import *

## Métodos de mínimos quadrados
Seja $\mathbf{A}\in\mathcal{R}^{m\times n}$. A decomposição QR tem como objetivo obter decompor a matriz $\mathbf{A}$ no produto $\mathbf{Q}\mathbf{R}$, tal que $\mathbf{Q}\in\mathcal{R}^{m\times n}$ é uma matriz ortogonal (isto é, $\mathbf{Q}^T \mathbf{Q} = \mathbf{I}$) e $\mathbf{R}\in\mathcal{R}^{n\times n}$ é uma matriz triangular superior, de modo que $\mathbf{A} = \mathbf{QR}$. A resolução de um sistema linear através da decomposição QR se dá da seguinte forma:
\begin{equation}
    \mathbf{A}\boldsymbol{x} = \boldsymbol{b} \iff \mathbf{QR}\boldsymbol{x} = \boldsymbol{b} \iff \mathbf{Q}^T\mathbf{QR}\boldsymbol{x} = \mathbf{Q}^T\boldsymbol{b} \iff \mathbf{IR}\boldsymbol{x} = \mathbf{Q}^T\boldsymbol{b} \iff \mathbf{R}\boldsymbol{x} = \mathbf{Q}^T\boldsymbol{b}.
\end{equation}
Como $\mathbf{R}$ é uma matriz de ordem $n$ triangular superior, o sistema pode ser resolvido retro-substituição.

### Gram-Schimidt
#### Definição
A decomposição de Gram-Schimidt infere a matriz $\mathbf{Q}$ a partir de $\mathbf{A}$. 

Defina $\mathcal{A}=\{\boldsymbol{a}_j: j=1,\dots,n\}$ e $\mathcal{Q}=\{\boldsymbol{q}_j: j=1,\dots,n\}$, o conjunto das colunas de $\mathbf{A}$ e $\mathbf{Q}$, respectivamente. Para determinar $\boldsymbol{q}_j\in\mathcal{Q}$, aplica-se a seguinte expressão
\begin{equation}
    \boldsymbol{q}_j = \frac{1}{||\boldsymbol{a}_j||_2}\left(\boldsymbol{a}_j - \sum_{k=1}^{j-1}\boldsymbol{a}_j^T\boldsymbol{q}_k\boldsymbol{q}_k\right).
\end{equation}

A matriz $\mathbf{R}$, por sua vez, é obtida pela seguinte relação
\begin{equation}
    \mathbf{A} = \mathbf{QR} \iff \mathbf{Q}^T\mathbf{A} = \mathbf{Q}^T\mathbf{QR} \iff \mathbf{Q}^T\mathbf{A} = \mathbf{R}.
\end{equation}


#### Código python
```Python
def gram_schmidt(self):
    N,M = self.shape
    V = self.copy()

    Q = zeros((N,N))
    R = zeros((N,M))

    for i in range(M):
        R[i,i] = V[:,i].norm()
        Q[:,i] = V[:,i] * (1/R[i,i])

        for j in range(i+1,M):
            R[i,j] = Q[:,i].transpose() * V[:,j]
            V[:,j] = V[:,j] - Q[:,i] * R[i,j]

    return Q,R
```

In [4]:
A = Matrix([[ 1,-1, 4],
            [ 1, 4,-2],
            [ 1, 4, 2],
            [ 1,-1, 0]])

Q, R = A.gram_schmidt()



print(Q)
print(R)
print(Q * R)


|    0.50    -0.50     0.50     0.00 |
|    0.50     0.50    -0.50     0.00 |
|    0.50     0.50     0.50     0.00 |
|    0.50    -0.50    -0.50     0.00 |


|    2.00     3.00     2.00 |
|    0.00     5.00    -2.00 |
|    0.00     0.00     4.00 |
|    0.00     0.00     0.00 |


|    1.00    -1.00     4.00 |
|    1.00     4.00    -2.00 |
|    1.00     4.00     2.00 |
|    1.00    -1.00     0.00 |



### Householder

#### Definição
Considere o mesmo problema acima, $\mathbf{Q}^T\mathbf{A} = \mathbf{R}$. No método de Householder, busca-se estimar matrizes ortogonais $\mathbf{H}_i\in\mathcal{R}^{m\times m}$, de modo que $\mathbf{H}_n\dots\mathbf{H}_2\mathbf{H}_1\mathbf{A} = \mathbf{R}$, em que
\begin{equation}
    \mathbf{H}_i = \mathbf{I} - 2\boldsymbol{n}_i\boldsymbol{n}_i^T.
\end{equation}
Para se obter o vetor $\boldsymbol{n}_i$, considere o vetor $\boldsymbol{v} = [0,\dots,a_{ij}, \dots, a_{mj}]^T$, tal que $v_j = v_j + (-\text{sinal}(v_j)||\boldsymbol{v}||_2)$, em que $\boldsymbol{a}_{:j}$ é a $j$-ésima coluna da matriz $\mathbf{A}$. O vetor $\boldsymbol{n}_i$ é então dado por $\boldsymbol{n}_i = \frac{\boldsymbol{v}}{||\boldsymbol{v}||_2}$.

Uma vez de posse da matriz $\mathbf{\overline{H}} = \mathbf{H}_n\dots\mathbf{H}_2\mathbf{H}_1$, defini-se a matriz $\mathbf{Q}^T=\{\overline{h}_{ij}:\forall i=1\dots,n, \forall j = 1,\dots,m\}$.


#### Código Python
```Python
def __make_householder_matrix__(self, A, j):
    N,M  = A.shape
    H  = eye(N)
    v  = zeros((N,1))
    v_ = zeros((N,1))

    v[j:N,0] = A[j:N,1]


    v_[j,0] = -((abs(v[j,0]) / v[j,0]) * v.norm())

    N_ = v - v_;

    n = N_ * (1/N_.norm())
    # print(n.transpose() * n)
    H = H - (n * n.transpose()) * 2

    return H

def householder_decomposition(self):
    N,M   = self.shape
    A_j = self.copy()
    H   = eye(N)
    for j in range(N):
        H_j = self.__make_householder_matrix__(A_j, j)
        A_j = H_j * A_j
        H   = H * H_j
    return H, A_j
```

In [6]:
A = Matrix([[ 1,-1, 4],
            [ 1, 4,-2],
            [ 1, 4, 2],
            [ 1,-1, 0]])

Q, R = A.householder_decomposition()
print(Q)
print(R)
print(Q * R)


|   -0.17    -0.98     0.06    -0.03 |
|    0.69    -0.11     0.43     0.57 |
|    0.69    -0.11    -0.20    -0.69 |
|   -0.17     0.10     0.88    -0.44 |


|    1.03     5.83    -0.69 |
|   -1.10     0.00    -3.93 |
|    1.18    -0.00    -1.01 |
|   -0.58     0.00    -2.66 |


|    1.00    -1.00     4.00 |
|    1.00     4.00    -2.00 |
|    1.00     4.00     2.00 |
|    1.00    -1.00     0.00 |

